In [21]:
print('Importing libraries and loading the trained model')
import magenta.music as mm
import note_seq
from note_seq import sequences_lib
from note_seq.protobuf import music_pb2
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
from magenta.models.music_vae.trained_model import NoExtractedExamplesError
import numpy as np
import os
import tensorflow.compat.v1 as tf
import random
import sqlite3
import numpy as np
import faiss  # You'll need to install this: pip install faiss-cpu



tf.disable_v2_behavior()

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.set_random_seed(SEED)
random.seed(SEED)

mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']

BASE_DIR="models/download.magenta.tensorflow.org/models/music_vae"
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/mel_2bar_big.ckpt')

Importing libraries and loading the trained model
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



d:\Users\thoma\Documents\git\python_midi_training\.venv\lib\site-packages\magenta\models\music_vae\lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
d:\Users\thoma\Documents\git\python_midi_training\.venv\lib\site-packages\magenta\contrib\rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
d:\Users\thoma\Documents\git\python_midi_training\.venv\lib\site-packages\magenta\contrib\rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(


INFO:tensorflow:Restoring parameters from models/download.magenta.tensorflow.org/models/music_vae/checkpoints/mel_2bar_big.ckpt


d:\Users\thoma\Documents\git\python_midi_training\.venv\lib\site-packages\magenta\models\music_vae\base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
d:\Users\thoma\Documents\git\python_midi_training\.venv\lib\site-packages\magenta\models\music_vae\base_model.py:200: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  sigma = tf.layers.dense(


In [3]:
# --- Configuration ---
DB_PATH = os.path.join('data_sets', 'midi_embeddings.db')
FAISS_INDEX_PATH = os.path.join('data_sets', 'midi_embeddings.index')
MELODY_DIR = os.path.join('data_sets', 'lmd_melodies') # Directory of extracted melodies
EMBEDDING_DIM = 512 # The dimension of your MusicVAE embeddings



In [4]:
def filter_pitch_range(ns, min_pitch=36, max_pitch=84):
    """Removes notes outside the specified MIDI pitch range."""
    valid_notes = [n for n in ns.notes if min_pitch <= n.pitch <= max_pitch]   
    del ns.notes[:]
    ns.notes.extend(valid_notes)   
    return ns

In [5]:
# --- Helper Functions (from previous iterations, still useful for cleaning) ---
def make_monophonic(ns,steps_per_quarter=4):
    """Reduces a NoteSequence to be monophonic by picking the highest note at each step."""
    if not ns.notes:
        return ns
    quantized_ns = sequences_lib.quantize_note_sequence(ns, steps_per_quarter)    
    notes_by_step = {}
    for note in quantized_ns.notes:
        # Use quantized_start_step for already quantized sequences
        if note.quantized_start_step not in notes_by_step:
            notes_by_step[note.quantized_start_step] = []
        notes_by_step[note.quantized_start_step].append(note)
    monophonic_notes = []
    for step in sorted(notes_by_step.keys()):
        notes_at_step = notes_by_step[step]
        # If multiple notes at a step, pick the highest pitch
        highest_note = max(notes_at_step, key=lambda n: n.pitch)
        monophonic_notes.append(highest_note)
    del ns.notes[:]
    ns.notes.extend(monophonic_notes)
    return ns

In [6]:
def snap_chunk_notes_to_grid(unquantized_chunk, steps_per_quarter):
    """
    Creates a new, unquantized NoteSequence with notes snapped to a grid.
    This is the key function. It takes a time-based chunk, finds the ideal
    quantized steps for its notes, and then creates a *new* unquantized
    sequence where the note start/end times correspond perfectly to those steps.
    Args:
      unquantized_chunk: The unquantized NoteSequence chunk.
      steps_per_quarter: The quantization resolution.
    Returns:
      A new, unquantized NoteSequence with grid-aligned note timings.
    """
    # 1. Quantize the chunk to determine the ideal grid steps for each note.
    try:
        quantized_temp_chunk = note_seq.quantize_note_sequence(
            unquantized_chunk, steps_per_quarter)
    except note_seq.BadTimeSignatureError:
        return None # Cannot process this chunk
    qpm = unquantized_chunk.tempos[0].qpm if unquantized_chunk.tempos else 120.0
    seconds_per_quarter = 60.0 / qpm
    # 2. Create a new, empty, unquantized sequence to be the output.
    grid_aligned_ns = music_pb2.NoteSequence()
    grid_aligned_ns.tempos.add().qpm = qpm
    grid_aligned_ns.ticks_per_quarter = unquantized_chunk.ticks_per_quarter
    # 3. For each note in the quantized version, create a new note in our
    #    output sequence with timings calculated from the quantized steps.
    for q_note in quantized_temp_chunk.notes:
        new_note = grid_aligned_ns.notes.add()
        new_note.pitch = q_note.pitch
        new_note.velocity = q_note.velocity
        new_note.instrument = q_note.instrument
        new_note.program = q_note.program
        # Convert quantized steps back into precise seconds
        start_quarters = q_note.quantized_start_step / steps_per_quarter
        end_quarters = q_note.quantized_end_step / steps_per_quarter
        new_note.start_time = start_quarters * seconds_per_quarter
        new_note.end_time = end_quarters * seconds_per_quarter
    # Set the total time of the new sequence.
    total_quarters = quantized_temp_chunk.total_quantized_steps / steps_per_quarter
    grid_aligned_ns.total_time = total_quarters * seconds_per_quarter
    return grid_aligned_ns


In [7]:
def set_program_for_all_notes(note_sequence, program_number=0):
    """
    Resets the instrument program for every note in a NoteSequence.
    Args:
      note_sequence: The note_seq.NoteSequence object to modify.
      program_number: The integer program number to set for all notes.
                      Defaults to 0 (Acoustic Grand Piano).
    Returns:
      The modified NoteSequence.
    """
    for note in note_sequence.notes:
        note.program = program_number
    return note_sequence


In [ ]:
config = configs.CONFIG_MAP['cat-mel_2bar_big']



# --- 2. Load the MIDI file (unquantized) ---
# Assume 'midi_ns' is your full NoteSequence object
# midi_ns = note_seq.midi_file_to_note_sequence(full_path)

# --- 3. Extract fixed-length, quantized chunks using sequences_lib.extract_subsequences ---
# This function expects an UNQUANTIZED sequence and will internally quantize it
# to extract segments of the specified number of steps.
num_steps_per_chunk = config.hparams.max_seq_len  # This will be 32 for mel_2bar
steps_per_quarter = config.data_converter._steps_per_quarter # This is needed by extract_subsequences internally

# --- 2. Load the MIDI file (unquantized) ---
full_path = 'data_sets/lmd_melodies/A/A/A/TRAAAGR128F425B14B/1d9d16a9da90c090809c153754823c2b.mid'
midi_ns = note_seq.midi_file_to_note_sequence(full_path)
cleaned_quantized_list = []
cleaned_ms = make_monophonic(midi_ns)
cleaned_ms = snap_chunk_notes_to_grid(cleaned_ms, steps_per_quarter)
cleaned_ms = set_program_for_all_notes(cleaned_ms, program_number=0)
qpm = cleaned_ms.tempos[0].qpm if cleaned_ms.tempos else 120.0
seconds_per_quarter = 60.0 / qpm
seconds_per_step = seconds_per_quarter / steps_per_quarter
hop_size_in_seconds = num_steps_per_chunk * seconds_per_step # 32 / 4 = 8.0 seconds
if cleaned_ms.notes:
   slices = sequences_lib.split_note_sequence(
        note_sequence=cleaned_ms,
        hop_size_seconds=hop_size_in_seconds  
)

print(f"The sequence was split into {len(slices)} slices.")

# You can now work with the 'slices' list, where each item is a 32-step NoteSequence.
if slices:
    print(f"The last quantized step of the first slice is: {slices[0].total_quantized_steps}")
    for chunk in slices:
        cleaned_quantized_list.append(chunk)
else:
    print(f"Melody became empty after cleaning, skipping.")
embeddings=[]
for chunk in cleaned_quantized_list:
    try:
        embedding = mel_2bar.encode([chunk])
        embeddings.append(embedding)
    except NoExtractedExamplesError as e:
        print(f"Skipping chunk due to error: {e}")
        continue
print(f"Generated {len(embeddings)} embeddings.")



The sequence was split into 54 slices.
The last quantized step of the first slice is: 0
Skipping chunk due to error: No examples extracted from NoteSequence: ticks_per_quarter: 220
tempos {
  qpm: 120.0
}
subsequence_info {
  start_time_offset: 48.0
  end_time_offset: 168.0
}

Skipping chunk due to error: No examples extracted from NoteSequence: ticks_per_quarter: 220
tempos {
  qpm: 120.0
}
subsequence_info {
  start_time_offset: 52.0
  end_time_offset: 164.0
}

Skipping chunk due to error: No examples extracted from NoteSequence: ticks_per_quarter: 220
tempos {
  qpm: 120.0
}
subsequence_info {
  start_time_offset: 56.0
  end_time_offset: 160.0
}

Skipping chunk due to error: No examples extracted from NoteSequence: ticks_per_quarter: 220
tempos {
  qpm: 120.0
}
subsequence_info {
  start_time_offset: 60.0
  end_time_offset: 156.0
}

Skipping chunk due to error: No examples extracted from NoteSequence: ticks_per_quarter: 220
tempos {
  qpm: 120.0
}
subsequence_info {
  start_time_off